In [1]:
!pip install pandas
!pip install boto3

In [2]:
import os
import numpy as np
import pandas as pd
from io import BytesIO
import boto3

In [10]:
diretorio_base='../../arquivos/exercicios/municipios-estados/csv'

In [31]:
# Conectando ao minio
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='oBazxeVyPCLX1D3g',
    aws_secret_access_key='dKJSfjsd74HbqWeYX1Th917Hfs4kDA2Q',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

### * Importar CSVs da pasta /exercicios/municipios-estados/csv/ e salvar como arquivos JSON no MinIO

In [52]:
# Gerando arquivos JSONs a partir de arquivos CSVs
for root,dirs,arquivos in os.walk(diretorio_base):
    dirs[:] = [d for d in dirs if not d.startswith('.')]
    for arquivo in arquivos:
        if arquivo.endswith('.csv'):
            df = pd.read_csv(os.path.join(diretorio_base, arquivo))
            # Converte o DataFrame para JSON
            json_data = df.to_json(orient='records', indent=4, force_ascii=False)
            nome_sem_extensao = os.path.splitext(arquivo)[0]
            caminho_json = os.path.join(diretorio_base, f'{nome_sem_extensao}.json')

            with open(caminho_json, 'w', encoding='utf-8') as f:
                f.write(json_data)

In [55]:
# Enviado para o MinIO
for root,dirs,arquivos in os.walk(diretorio_base):
    dirs[:] = [d for d in dirs if not d.startswith('.')]
    for arquivo in arquivos:
        if arquivo.endswith('.json'):
            caminho_completo = os.path.join(root, arquivo)
            caminho_minio = os.path.relpath(caminho_completo, diretorio_base)
            parts = [part for part in caminho_minio.split(os.sep) if part not in (".", "..")]
            caminho_minio = "/".join(parts)

            try:
                client.upload_file(
                    caminho_completo, 'aula-08', caminho_minio
                )
                print(f"Arquivo '{caminho_minio}' enviado para o MinIO.")
            except Exception as err:
                print(f"Erro ao enviar '{caminho_minio}': {err}")

Arquivo 'estados.json' enviado para o MinIO.
Arquivo 'municipios.json' enviado para o MinIO.


### * Importar JSONs da pasta de /exercicios/municipios-estados/csv/ e salvar como arquivos CSV no MinIO

In [73]:
# Gerando arquivos CSVs a partir de arquivos JSONs
for arquivo in os.listdir(diretorio_base):
    if arquivo.endswith('.json'):
        df = pd.read_json(os.path.join(diretorio_base, arquivo))
        csv_data = df.to_csv(index=False)
        nome_sem_extensao = os.path.splitext(arquivo)[0]
        caminho_csv = os.path.join(diretorio_base, f'{nome_sem_extensao}.csv')

        with open(caminho_csv, 'w', encoding='utf-8') as f:
            f.write(csv_data)
    

In [74]:
# Enviado para o MinIO
for arquivo in os.listdir(diretorio_base):
    if arquivo.endswith('.csv'):
        caminho_completo = os.path.join(diretorio_base, arquivo)
        caminho_minio = os.path.relpath(caminho_completo, diretorio_base)
        parts = [part for part in caminho_minio.split(os.sep) if part not in (".", "..")]
        caminho_minio = "/".join(parts)
        
        try:
            client.upload_file(caminho_completo, 'aula-08', caminho_minio)
            print(f"Arquivo '{caminho_minio}' enviado para o MinIO.")
        except Excepion as err:
            print(f"Erro ao enviar '{caminho_minio}': '{err}'")
        

Arquivo 'estados.csv' enviado para o MinIO.
Arquivo 'municipios.csv' enviado para o MinIO.


### * Importar CSVs da pasta de /exercicios/municipios-estados/csv/ e salvar como arquivo .parquet no MinIO

In [84]:
# Gerando arquivos PARQUETs a partir de arquivos CSVs
for arquivo in os.listdir(diretorio_base):
    if arquivo.endswith('.csv'):
        df = pd.read_csv(os.path.join(diretorio_base,arquivo))
        nome_sem_extensao = os.path.splitext(arquivo)[0]
        caminho_parquet = os.path.join(diretorio_base, f'{nome_sem_extensao}.parquet')
        parquet_data = df.to_parquet(engine='pyarrow')
        
        with open(caminho_parquet, 'wb') as f:
            f.write(parquet_data)
            

In [85]:
# Enviado para o MinIO
for arquivo in os.listdir(diretorio_base):
    if arquivo.endswith('.parquet'):
        caminho_completo = os.path.join(diretorio_base, arquivo)
        caminho_minio = os.path.relpath(caminho_completo, diretorio_base)
        parts = [part for part in caminho_minio.split(os.sep) if part not in (".", "..")]
        caminho_minio = "/".join(parts)
        
        
        try:
            client.upload_file(caminho_completo, 'aula-08', caminho_minio)
            print(f"Arquivo '{caminho_minio}' enviado para o MinIO.")
        except Excepion as err:
            print(f"Erro ao enviar '{caminho_minio}': '{err}'")

Arquivo 'estados.parquet' enviado para o MinIO.
Arquivo 'municipios.parquet' enviado para o MinIO.
